# Characterizing CpG Methylation

To describe general metylation trends, irrespective of pCO<sub>2</sub> treatment in *C. virginica* gonad sequence data, I need to characterize individual CpG loci. Gavery and Roberts (2013) and Olson and Roberts (2013) define a CpG locus as methylated if at least half of the reads remained unconverted after bisulfite treatment. I will use information in a master `.cov` file to identify methylated CpG loci.

1. Download coverage file
2. Limit to 5x coverage
3. Identify methylated loci
4. Characterize location of methylated loci

## 0. Prepare for analyses

## 0a. Set working directory

In [1]:
pwd

'/Users/yaamini/Documents/yaamini-virginica/notebooks'

In [2]:
cd ../analyses/

/Users/yaamini/Documents/yaamini-virginica/analyses


In [3]:
!mkdir 2019-03-18-Characterizing-CpG-Methylation

In [3]:
cd 2019-03-18-Characterizing-CpG-Methylation/

/Users/yaamini/Documents/yaamini-virginica/analyses/2019-03-18-Characterizing-CpG-Methylation


## 1. Obtain coverage files

In [4]:
#Download file from gannet. This file is a concatenation of coverage and methylation information for all samples
!wget http://gannet.fish.washington.edu/Atumefaciens/20190312_cvir_gonad_bismark/total_reads_bismark/cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov.gz

--2019-04-09 14:41:39--  http://gannet.fish.washington.edu/Atumefaciens/20190312_cvir_gonad_bismark/total_reads_bismark/cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov.gz
Resolving gannet.fish.washington.edu... 128.95.149.52
Connecting to gannet.fish.washington.edu|128.95.149.52|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94181669 (90M) [application/x-gzip]
Saving to: 'cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov.gz'

cvir_bsseq_all_pe_R 100%[===================>]  89.82M  75.1MB/s    in 1.2s    

2019-04-09 14:41:40 (75.1 MB/s) - 'cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov.gz' saved [94181669/94181669]



In [8]:
#Unzip the coverage file
!gunzip *cov.gz

In [9]:
#Confirm file was unzipped
!ls *cov

cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov


In [4]:
#See what the file looks like. 
#Columns: <chromosome> <start position> <end position> <methylation percentage> <count methylated> <count unmethylated>
!head cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov

NC_007175.2	49	49	1.25	2	158
NC_007175.2	50	50	0	0	15
NC_007175.2	51	51	1.18343195266272	2	167
NC_007175.2	52	52	0	0	18
NC_007175.2	88	88	1.02459016393443	5	483
NC_007175.2	89	89	1.38888888888889	5	355
NC_007175.2	100	100	0	0	1
NC_007175.2	129	129	0	0	1
NC_007175.2	147	147	1.99115044247788	18	886
NC_007175.2	148	148	2.29885057471264	6	255


In [6]:
#See how many loci have data
!awk '{if ($5+$6 >= 1) { print $1, $2-1, $3, $4, $5+$6}}' cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov \
| wc -l

 14026131


14,026,131 CpGs have data, which is close to the 14,458,703 CG motifs in the *C. virginica* genome.

## 2. Limit to 5x coverage

In [11]:
#If total coverage (count methylated + unmethylated) is greater than 5
#then print the chromosome, start pos -1, stop pos, percent methylation, and total coverage
#Save output as new file
!awk '{if ($5+$6 >= 5) { print $1, $2-1, $3, $4, $5+$6}}' cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov \
> 2019-04-09-All-5x-CpGs.bedgraph

In [12]:
#Check columns for one of the file: <chromosome> <start position> <stop position> <percent methylation> <coverage>
!head 2019-04-09-All-5x-CpGs.bedgraph

NC_007175.2 48 49 1.25 160
NC_007175.2 49 50 0 15
NC_007175.2 50 51 1.18343195266272 169
NC_007175.2 51 52 0 18
NC_007175.2 87 88 1.02459016393443 488
NC_007175.2 88 89 1.38888888888889 360
NC_007175.2 146 147 1.99115044247788 904
NC_007175.2 147 148 2.29885057471264 261
NC_007175.2 173 174 0 5
NC_007175.2 192 193 1.25786163522013 795


In [58]:
#Count loci with 5x coverage
!wc -l 2019-04-09-All-5x-CpGs.bedgraph

 4304257 2019-04-09-All-5x-CpGs.bedgraph


I have data for 4,304,257 CpG loci with 5x coverge.

In [59]:
#Replace delimiters to save .bedgraph as .csv
!awk '{print $1","$2","$3","$4 }' 2019-04-09-All-5x-CpGs.bedgraph \
> 2019-04-09-All-5x-CpGs.csv

In [60]:
#Confirm .csv creation
!head 2019-04-09-All-5x-CpGs.csv

NC_007175.2,48,49,1.25
NC_007175.2,49,50,0
NC_007175.2,50,51,1.18343195266272
NC_007175.2,51,52,0
NC_007175.2,87,88,1.02459016393443
NC_007175.2,88,89,1.38888888888889
NC_007175.2,146,147,1.99115044247788
NC_007175.2,147,148,2.29885057471264
NC_007175.2,173,174,0
NC_007175.2,192,193,1.25786163522013


## 3. Identify methylated loci

Olson and Roberts (2014) define the following categories for CpG methylation:

- Methylated (50% methylation and above)
- Sparsely methylated (0-50% methylated)
- Unmethylated (0% methylation)

I will slightly modify this since I have multiple samples:

- Methylated (50% methylation and above)
- Sparsely methylated (10-50% methylated)
- Unmethylated (10% methylation and below)

### 3a. Methylated loci

In [19]:
#If percent methylation is greater or equal to 50, then save the loci information
!awk '{if ($4 >= 50) { print $1, $2, $3, $4 }}' 2019-04-09-All-5x-CpGs.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph

In [48]:
#Confirm methylated loci were saved
!head 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph

NC_035780.1 9253 9254 60
NC_035780.1 9637 9638 60
NC_035780.1 9657 9658 50
NC_035780.1 10089 10090 71.4285714285714
NC_035780.1 10331 10332 80
NC_035780.1 11692 11693 80
NC_035780.1 11706 11707 80
NC_035780.1 11711 11712 80
NC_035780.1 12686 12687 69.2307692307692
NC_035780.1 12758 12759 80


In [21]:
#Count methylated loci
!wc -l 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph

 3181904 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph


In [53]:
#Replace delimiters to save .bedgraph as .csv
!awk '{print $1","$2","$3","$4 }' 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Methylated.csv

In [54]:
#Check .csv was saved
!head 2019-04-09-All-5x-CpG-Loci-Methylated.csv

NC_035780.1,9253,9254,60
NC_035780.1,9637,9638,60
NC_035780.1,9657,9658,50
NC_035780.1,10089,10090,71.4285714285714
NC_035780.1,10331,10332,80
NC_035780.1,11692,11693,80
NC_035780.1,11706,11707,80
NC_035780.1,11711,11712,80
NC_035780.1,12686,12687,69.2307692307692
NC_035780.1,12758,12759,80


### 3b. Sparsely methylated loci

In [33]:
%%bash
awk '{if ($4 < 50) { print $1, $2, $3, $4}}' 2019-04-09-All-5x-CpGs.bedgraph \
| awk '{if ($4 > 10) { print $1, $2, $3, $4 }}' \
> 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bedgraph

In [34]:
#Confirm sparsely methylated loci were saved
!head 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bedgraph

NC_007175.2 1506 1507 16.6666666666667
NC_007175.2 1820 1821 20
NC_007175.2 2128 2129 11.7647058823529
NC_007175.2 4841 4842 15
NC_007175.2 13069 13070 20
NC_035780.1 421 422 14.2857142857143
NC_035780.1 1101 1102 12.5
NC_035780.1 1540 1541 16.6666666666667
NC_035780.1 3468 3469 16.6666666666667
NC_035780.1 9254 9255 28.5714285714286


In [35]:
#Count sparsely methylated loci
!wc -l 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bedgraph

  481788 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bedgraph


### 3c. Unmethylated loci

In [36]:
!awk '{if ($4 <= 10) { print $1, $2, $3, $4 }}' 2019-04-09-All-5x-CpGs.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Unmethylated.bedgraph

In [37]:
#Confirm unmethylated loci were saved
!head 2019-04-09-All-5x-CpG-Loci-Unmethylated.bedgraph

NC_007175.2 48 49 1.25
NC_007175.2 49 50 0
NC_007175.2 50 51 1.18343195266272
NC_007175.2 51 52 0
NC_007175.2 87 88 1.02459016393443
NC_007175.2 88 89 1.38888888888889
NC_007175.2 146 147 1.99115044247788
NC_007175.2 147 148 2.29885057471264
NC_007175.2 173 174 0
NC_007175.2 192 193 1.25786163522013


In [38]:
#Count unmethylated loci
!wc -l 2019-04-09-All-5x-CpG-Loci-Unmethylated.bedgraph

  640565 2019-04-09-All-5x-CpG-Loci-Unmethylated.bedgraph


## 4. Location of methylated loci

My final step is to characterize the location of methylated loci in the genome. I will use `intersectBed` to find overlaps between methylated loci and exons, introns, mRNA coding regions, transposable elements, and putative promoter regions.

### 4a. Created `.bed` file

In [8]:
%%bash
awk '{print $1"\t"$2"\t"$3}' 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Methylated.bed

In [47]:
#Confirm file creation
!head 2019-04-09-All-5x-CpG-Loci-Methylated.bed

NC_035780.1	9253	9254
NC_035780.1	9637	9638
NC_035780.1	9657	9658
NC_035780.1	10089	10090
NC_035780.1	10331	10332
NC_035780.1	11692	11693
NC_035780.1	11706	11707
NC_035780.1	11711	11712
NC_035780.1	12686	12687
NC_035780.1	12758	12759


### 4b. Set variable paths

In [11]:
bedtoolsDirectory = "/Users/Shared/bioinformatics/bedtools2/bin/"

In [12]:
methylatedLoci = "2019-04-09-All-5x-CpG-Loci-Methylated.bed"

In [13]:
exonList = "../2018-11-01-DML-and-DMR-Analysis/C_virginica-3.0_Gnomon_exon.bed"

In [14]:
intronList = "../2018-11-01-DML-and-DMR-Analysis/C_virginica-3.0_intron.bed"

In [15]:
mRNAList = "../2018-11-01-DML-and-DMR-Analysis/C_virginica-3.0_Gnomon_mRNA.gff3"

In [16]:
transposableElementsAll = "../2018-11-01-DML-and-DMR-Analysis/C_virginica-3.0_TE-all.gff"

In [17]:
transposableElementsCg = "../2018-11-01-DML-and-DMR-Analysis/C_virginica-3.0_TE-Cg.gff"

In [18]:
putativePromoters = "../2018-11-01-DML-and-DMR-Analysis/2018-11-14-Flanking-Analysis/2018-11-15-mRNA-Upstream-Flanks.bed"

### 4c. Exons

In [19]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {exonList} \
| wc -l
!echo "methylated loci overlaps with exons"

 1013691
methylated loci overlaps with exons


In [20]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {exonList} \
> 2019-04-10-MethLoci-Exon.txt

In [21]:
!head 2019-04-10-MethLoci-Exon.txt

NC_035780.1	100558	100559	NC_035780.1	100554	100661
NC_035780.1	100559	100560	NC_035780.1	100554	100661
NC_035780.1	100575	100576	NC_035780.1	100554	100661
NC_035780.1	100576	100577	NC_035780.1	100554	100661
NC_035780.1	100581	100582	NC_035780.1	100554	100661
NC_035780.1	100582	100583	NC_035780.1	100554	100661
NC_035780.1	100634	100635	NC_035780.1	100554	100661
NC_035780.1	100635	100636	NC_035780.1	100554	100661
NC_035780.1	100643	100644	NC_035780.1	100554	100661
NC_035780.1	100644	100645	NC_035780.1	100554	100661


### 4d. Introns

In [22]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {intronList} \
| wc -l
!echo "methylated loci overlaps with introns"

 1448786
methylated loci overlaps with introns


In [23]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {intronList} \
> 2019-04-10-MethLoci-Intron.txt

In [24]:
!head 2019-04-10-MethLoci-Intron.txt

NC_035780.1	29412	29413	NC_035780.1	29074	30524
NC_035780.1	87531	87532	NC_035780.1	85778	88423
NC_035780.1	87541	87542	NC_035780.1	85778	88423
NC_035780.1	87590	87591	NC_035780.1	85778	88423
NC_035780.1	87595	87596	NC_035780.1	85778	88423
NC_035780.1	100664	100665	NC_035780.1	100662	104929
NC_035780.1	100665	100666	NC_035780.1	100662	104929
NC_035780.1	100917	100918	NC_035780.1	100662	104929
NC_035780.1	100975	100976	NC_035780.1	100662	104929
NC_035780.1	101305	101306	NC_035780.1	100662	104929


### 4e. mRNA

In [25]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {mRNAList} \
| wc -l
!echo "methylated loci overlaps with mRNA coding regions"

 2437901
methylated loci overlaps with mRNA coding regions


In [26]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {mRNAList} \
> 2019-04-10-MethLoci-mRNA.txt

In [31]:
!head -n 1 2019-04-10-MethLoci-mRNA.txt

NC_035780.1	29412	29413	NC_035780.1	Gnomon	mRNA	28961	33324	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1


In [32]:
!cut -f12 2019-04-10-MethLoci-mRNA.txt| sort | uniq -c > 2019-04-10-Unique-Genes-in-MethLoci-mRNA-Overlap.txt

In [33]:
!head -n 1 2019-04-10-Unique-Genes-in-MethLoci-mRNA-Overlap.txt

  95 ID=rna10000;Parent=gene5866;Dbxref=GeneID:111121983,Genbank:XM_022463489.1;Name=XM_022463489.1;gbkey=mRNA;gene=LOC111121983;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 3 samples with support for all annotated introns;product=sodium-coupled neutral amino acid transporter 9-like%2C transcript variant X4;transcript_id=XM_022463489.1


In [34]:
!wc -l 2019-04-10-Unique-Genes-in-MethLoci-mRNA-Overlap.txt

   44505 2019-04-10-Unique-Genes-in-MethLoci-mRNA-Overlap.txt


Methylated loci overlap with 44,505 unique genes.

### 4f. Transposable elements (all)

In [35]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {transposableElementsAll} \
| wc -l
!echo "methylated loci overlaps with transposable elements (all)"

  755222
methylated loci overlaps with transposable elements (all)


In [36]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {transposableElementsAll} \
> 2019-04-10-MethLoci-TE-All.txt

In [37]:
!head 2019-04-10-MethLoci-TE-All.txt

NC_035780.1	9253	9254	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	19631	19632	NC_035780.1	RepeatMasker	similarity	19431	19866	23.3	-	.	Target "Motif:Crypton-N19_CGi" 580 1033
NC_035780.1	19741	19742	NC_035780.1	RepeatMasker	similarity	19431	19866	23.3	-	.	Target "Motif:Crypton-N19_CGi" 580 1033
NC_035780.1	37557	37558	NC_035780.1	RepeatMasker	similarity	37557	37890	12.9	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	37581	37582	NC_035780.1	RepeatMasker	similarity	37557	37890	12.9	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	37604	37605	NC_035780.1	RepeatMasker	similarity	37557	37890	12.9	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	37611	37612	NC_035780.1	RepeatMasker	similarity	37557	37890	12.9	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	37618	37619	NC_035780.1	RepeatMasker	similarity	37557	37890	12.9	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	37622	37623	NC_035780.1	RepeatMasker	

### 4g. Transposable elements (*C. gigas* only)

In [38]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {transposableElementsCg} \
| wc -l
!echo "methylated loci overlaps with transposable elements (Cg)"

  610208
methylated loci overlaps with transposable elements (Cg)


In [39]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {transposableElementsCg} \
> 2019-04-10-MethLoci-TE-Cg.txt

In [40]:
!head 2019-04-10-MethLoci-TE-Cg.txt

NC_035780.1	9253	9254	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	19631	19632	NC_035780.1	RepeatMasker	similarity	19431	19866	23.3	-	.	Target "Motif:Crypton-N19_CGi" 580 1033
NC_035780.1	19741	19742	NC_035780.1	RepeatMasker	similarity	19431	19866	23.3	-	.	Target "Motif:Crypton-N19_CGi" 580 1033
NC_035780.1	41723	41724	NC_035780.1	RepeatMasker	similarity	41713	41751	10.3	+	.	Target "Motif:Helitron-N10B_CGi" 258 296
NC_035780.1	41723	41724	NC_035780.1	RepeatMasker	similarity	41719	41776	 6.9	+	.	Target "Motif:Helitron-10_CGi" 282 358
NC_035780.1	73023	73024	NC_035780.1	RepeatMasker	similarity	72892	73822	28.6	-	.	Target "Motif:Kolobok-N4_CGi" 1 925
NC_035780.1	87531	87532	NC_035780.1	RepeatMasker	similarity	87526	87837	24.3	-	.	Target "Motif:DNA3-12_CGi" 60 378
NC_035780.1	87541	87542	NC_035780.1	RepeatMasker	similarity	87526	87837	24.3	-	.	Target "Motif:DNA3-12_CGi" 60 378
NC_035780.1	87590	87591	NC_035780.1	RepeatMasker	similarity	

### 4h. Putative promoters

In [41]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {putativePromoters} \
| wc -l
!echo "methylated loci overlaps with putative promoters"

  134534
methylated loci overlaps with putative promoters


In [42]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {putativePromoters} \
> 2019-04-10-MethLoci-Putative-Promoters.txt

In [43]:
!head 2019-04-10-MethLoci-Putative-Promoters.txt

NC_035780.1	27969	27970	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	27979	27980	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	28082	28083	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:1

### 4i. No overlaps

In [44]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {methylatedLoci} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
| wc -l
!echo "methylated loci do not overlap with exons, introns, transposable elements (all), or putative promoters"

  386003
methylated loci do not overlap with exons, introns, transposable elements (all), or putative promoters


In [45]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {methylatedLoci} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
> 2019-04-10-MethLoci-NoOverlaps.txt

In [46]:
!head 2019-04-10-MethLoci-NoOverlaps.txt

NC_035780.1	9637	9638
NC_035780.1	9657	9658
NC_035780.1	10089	10090
NC_035780.1	10331	10332
NC_035780.1	11692	11693
NC_035780.1	11706	11707
NC_035780.1	11711	11712
NC_035780.1	12686	12687
NC_035780.1	12758	12759
NC_035780.1	13486	13487
